In [38]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint

In [21]:
train_dir = 'ClearDataset/tandur_data/train'
val_dir = 'ClearDataset/tandur_data/valid'

In [22]:
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalisasi gambar
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [23]:
val_datagen = ImageDataGenerator(rescale=1./255)

In [24]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Ukuran input untuk MobileNet
    batch_size=32,
    class_mode='categorical'  # untuk klasifikasi multi-kelas
)

Found 46358 images belonging to 25 classes.


In [25]:
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 11588 images belonging to 25 classes.


In [26]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Menambahkan lapisan-lapisan baru di atas base model
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)  # Global Average Pooling layer
x = layers.Dense(1024, activation='relu')(x)  # Lapisan dense
x = layers.Dropout(0.5)(x)  # Dropout untuk regularisasi
predictions = layers.Dense(train_generator.num_classes, activation='softmax')(x)  # Output layer

In [27]:
model = models.Model(inputs=base_model.input, outputs=predictions)

In [28]:
for layer in base_model.layers:
    layer.trainable = False

In [29]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [30]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)

/home/wan/anaconda3/envs/tf2/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1448/1448 ━━━━━━━━━━━━━━━━━━━━ 482s 321ms/step - accuracy: 0.7157 - loss: 0.9255 - val_accuracy: 0.8795 - val_loss: 0.3480
Epoch 2/10
   1/1448 ━━━━━━━━━━━━━━━━━━━━ 44s 31ms/step - accuracy: 0.8438 - loss: 0.5315

/home/wan/anaconda3/envs/tf2/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


1448/1448 ━━━━━━━━━━━━━━━━━━━━ 1s 657us/step - accuracy: 0.8438 - loss: 0.5315 - val_accuracy: 0.5000 - val_loss: 2.4285
Epoch 3/10
1448/1448 ━━━━━━━━━━━━━━━━━━━━ 297s 205ms/step - accuracy: 0.8557 - loss: 0.4270 - val_accuracy: 0.8998 - val_loss: 0.2899
Epoch 4/10
1448/1448 ━━━━━━━━━━━━━━━━━━━━ 0s 6us/step - accuracy: 0.9688 - loss: 0.1130 - val_accuracy: 0.7500 - val_loss: 0.2358
Epoch 5/10
1448/1448 ━━━━━━━━━━━━━━━━━━━━ 305s 210ms/step - accuracy: 0.8706 - loss: 0.3859 - val_accuracy: 0.9210 - val_loss: 0.2361
Epoch 6/10
1448/1448 ━━━━━━━━━━━━━━━━━━━━ 0s 7us/step - accuracy: 0.8438 - loss: 0.3403 - val_accuracy: 1.0000 - val_loss: 0.0013
Epoch 7/10


2024-12-04 01:26:14.649036: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 3140085843291211997
2024-12-04 01:26:14.649085: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 11721142309957638537


1448/1448 ━━━━━━━━━━━━━━━━━━━━ 304s 210ms/step - accuracy: 0.8754 - loss: 0.3679 - val_accuracy: 0.9062 - val_loss: 0.2661
Epoch 8/10
1448/1448 ━━━━━━━━━━━━━━━━━━━━ 0s 16us/step - accuracy: 0.9375 - loss: 0.1895 - val_accuracy: 1.0000 - val_loss: 0.0134
Epoch 9/10


2024-12-04 01:31:18.741806: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 11721142309957638537


1448/1448 ━━━━━━━━━━━━━━━━━━━━ 349s 241ms/step - accuracy: 0.8824 - loss: 0.3482 - val_accuracy: 0.9224 - val_loss: 0.2284
Epoch 10/10
1448/1448 ━━━━━━━━━━━━━━━━━━━━ 0s 7us/step - accuracy: 0.9375 - loss: 0.2172 - val_accuracy: 0.7500 - val_loss: 0.5806


2024-12-04 01:37:07.914855: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 3140085843291211997
2024-12-04 01:37:07.914902: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 11721142309957638537


In [31]:
score = model.evaluate(val_generator)
print(f"Loss: {score[0]}, Accuracy: {score[1]}")

363/363 ━━━━━━━━━━━━━━━━━━━━ 17s 47ms/step - accuracy: 0.9213 - loss: 0.2328
Loss: 0.22927053272724152, Accuracy: 0.9225923418998718


In [50]:
model.save('mobilenet_model.keras')

In [52]:
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
checkpoint = ModelCheckpoint('mobilenet_model.keras', monitor='val_loss', save_best_only=True)

In [53]:
for layer in base_model.layers[-100:]:
    layer.trainable = True


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history_finetune = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    callbacks=[early_stopping, lr_scheduler, checkpoint]
)

Epoch 1/20


2024-12-04 01:46:59.335664: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_11219', 104 bytes spill stores, 104 bytes spill loads



 640/1448 ━━━━━━━━━━━━━━━━━━━━ 2:38 197ms/step - accuracy: 0.6687 - loss: 1.1898

In [19]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

NameError: name 'plt' is not defined